In [0]:
%sql
-- drop table if exists silver.products_silver;
-- drop table if exists silver.brands_silver;
-- drop table if exists gold.dim_audit;
-- drop table if exists gold.dim_product;
drop table if exists gold.dim_customer;
-- drop table if exists gold.fact_sales;
-- drop table if exists gold.dim_brand;
-- drop table if exists gold.dim_calendar;
-- drop table if exists silver.sales_silver;
-- drop table if exists default.products_silver;
-- drop table if exists default.dim_audit;
-- drop table if exists silver.dim_audit;
-- drop table if exists silver.brands_silver;
-- drop table if exists silver.customers_silver;


In [0]:
%sql
create database if not exists silver;
create database if not exists gold;

In [0]:
%sql
create table if not exists silver.products_silver
(
  ProductId bigint generated always as identity,
  product_code string not null,
  product_name string not null,
  category string,
  description string,
  gender string,
  color string,
  size string,
  file_path string not null,
  file_size long not null,
  file_modification_time timestamp not null,
  active_status string not null,
  valid_from date not null,
  valid_to date not null
)
using delta
location '/FileStore/mnt/test_bucket/silver/products_silver'
TBLPROPERTIES(delta.enableChangeDataFeed = true);

In [0]:
%sql
create table if not exists silver.brands_silver
(
  BrandId bigint generated always as identity,
  brand_code string not null,
  name string not null,
  contact string,
  country string,
  file_path string not null,
  file_size long,
  file_modification_time timestamp,
  active_status string not null,
  valid_from date not null,
  valid_to date not null
)
using delta
location '/FileStore/mnt/test_bucket/silver/brands_silver'
TBLPROPERTIES(delta.enableChangeDataFeed = true);

In [0]:
%sql
insert into silver.brands_silver (brand_code, name, contact, country, file_path, active_status, valid_from, valid_to)
values('UNKN', 'not applicable', 'not applicable', 'not applicable', 'not applicable', 'Y', '2023-01-01', '9999-12-31')

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
create table if not exists silver.customers_silver
(
  CustomerId bigint generated always as identity,
  customer_code string not null,
  name string not null,
  surname string not null,
  email  string not null,
  country string,
  city string,
  address string,
  county string,
  state string,
  zip string,
  file_path string not null,
  file_size long,
  file_modification_time timestamp,
  active_status string not null,
  valid_from date not null,
  valid_to date not null
)
using delta
location '/FileStore/mnt/test_bucket/silver/customers_silver'
TBLPROPERTIES(delta.enableChangeDataFeed = true);

In [0]:
%sql
insert into silver.customers_silver (customer_code, name, surname, email, country, city, address, county, state, zip, file_path, active_status, valid_from, valid_to)
values('UNKN', 'not applicable', 'not applicable', 'not applicable', 'not applicable', 'not applicable', 'not applicable', 'not applicable', 'not applicable', 'not applicable', 'not applicable', 'Y', '2023-01-01', '9999-12-31')

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
create table if not exists silver.sales_silver
(
  Id bigint generated always as identity,
  DateKey string not null,
  ProductId bigint not null,
  BrandId bigint not null,
  CustomerId bigint not null,
  AuditId  bigint not null,
  order_number bigint not null,
  order_line int not null,
  price float not null,
  qty int not null,
  discount_perc float not null,
  cost float not null,
  file_path string not null,
  file_size long not null,
  file_modification_time timestamp not null
)
using delta
location '/FileStore/mnt/test_bucket/silver/sales_silver'
TBLPROPERTIES(delta.enableChangeDataFeed = true);

In [0]:
%sql
create table if not exists silver.dim_audit
(
  AuditId bigint generated always as identity,
  etl_version string not null,
  notebook_name string not null,
  user string not null,
  numb_of_rows_processed int not null,
  numb_of_rows_read int not null,
  files_read int not null,
  run_time timestamp,
  seconds_taken float
)
using delta
location '/FileStore/mnt/test_bucket/gold/dim_audit'
TBLPROPERTIES(delta.enableChangeDataFeed = true);

In [0]:
%sql
create table if not exists gold.dim_product
(
  ProductId bigint not null,
  product_code string not null,
  product_name string not null,
  category string,
  description string,
  gender string,
  color string,
  size string,
  active_status string not null,
  valid_from date not null,
  valid_to date not null,
  _commit_version long not null
)
using delta
location '/FileStore/mnt/test_bucket/gold/dim_product';

In [0]:
%sql
create table if not exists gold.dim_brand
(
  BrandId  bigint not null,
  brand_code string not null,
  name string not null,
  contact string,
  country string,
  active_status string not null,
  valid_from date not null,
  valid_to date not null,
  _commit_version long not null
)
using delta
location '/FileStore/mnt/test_bucket/gold/dim_brand';

In [0]:
%sql
create table if not exists gold.dim_customer
(
  CustomerId bigint not null,
  customer_code string not null,
  name string not null,
  surname string not null,
  email  string not null,
  country string,
  city string,
  address string,
  county string,
  state string,
  zip string,
  active_status string not null,
  valid_from date not null,
  valid_to date not null,
  _commit_version long not null
)
using delta
location '/FileStore/mnt/test_bucket/gold/dim_customer';

In [0]:
%sql
create table if not exists gold.fact_sales
(
  Id  bigint not null,
  ProductId bigint not null,
  BrandId bigint not null,
  CustomerId bigint not null,
  AuditId bigint not null,
  order_number bigint not null,
  order_line int not null,
  price float not null,
  qty int not null,
  discount_perc float not null,
  cost float not null,
  total_amount float not null,
  total_amount_with_discount float not null,
  _commit_version long not null
)
using delta
location '/FileStore/mnt/test_bucket/gold/fact_sales';

In [0]:
%sql
create table if not exists gold.dim_calendar
using delta
location '/FileStore/mnt/test_bucket/gold/dim_calendar';

COPY INTO gold.dim_calendar
  FROM '/FileStore/mnt/test_bucket/archive/000_calendar-2.csv'
  FILEFORMAT = CSV
  FORMAT_OPTIONS ('mergeSchema' = 'true',
                  'delimiter' = ',',
                  'header' = 'true')
  COPY_OPTIONS ('mergeSchema' = 'true');

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
1096,1096,0


In [0]:
spark.read.table('gold.dim_calendar').printSchema()

root
 |-- Date_Key: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Day_Number: string (nullable = true)
 |-- Month_Number: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- MMM_YYYY: string (nullable = true)
 |-- Day_Name: string (nullable = true)
 |-- Is_weekend: string (nullable = true)

